# Customer Segmentation Using RFM Model

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing

import time, warnings
import datetime as dt

#visualizations
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline
import seaborn as sns

warnings.filterwarnings("ignore")

## Get the Data

In [2]:
#load the dataset
retail_df = pd.read_csv("Sales_Data1.csv")
retail_df.tail()

,sl_no,Invoice,StockCode,Description,Quantity,InvoiceDate,Date,Time,Price,Customer ID,Country,Total_price,gender,Age
824359,824359,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12-09-2011 12:50,09-12-2011,12:50:00,2.10,12680,France,12.60,male,65
824360,824360,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12-09-2011 12:50,09-12-2011,12:50:00,4.15,12680,France,16.60,male,65
824361,824361,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12-09-2011 12:50,09-12-2011,12:50:00,4.15,12680,France,16.60,male,65
824362,824362,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12-09-2011 12:50,09-12-2011,12:50:00,4.95,12680,France,14.85,male,65
824363,824363,581587,POST,POSTAGE,1,12-09-2011 12:50,09-12-2011,12:50:00,18.00,12680,France,18.00,male,65


## Prepare the Data

In [3]:
retail_uk = retail_df[retail_df['Country']=='United Kingdom']
#check the shape
retail_uk.shape

(741301, 14)

In [4]:
#remove canceled orders
retail_uk = retail_uk[retail_uk['Quantity']>0]
retail_uk.shape

(741301, 14)

In [5]:
#remove rows where customerID are NA
retail_uk.dropna(subset=['Customer ID'],how='all',inplace=True)
retail_uk.shape

(741301, 14)

In [6]:
#restrict the data to one full year because it's better to use a metric per Months or Years in RFM
retail_uk = retail_uk[retail_uk['InvoiceDate']>= "2010-01-01"]
retail_uk.shape

(233932, 14)

In [7]:
print("Summary..")
#exploring the unique values of each attribute
print("Number of transactions: ", retail_uk['Invoice'].nunique())
print("Number of products bought: ",retail_uk['StockCode'].nunique())
print("Number of customers:", retail_uk['Customer ID'].nunique() )
print("Percentage of customers NA: ", round(retail_uk['Customer ID'].isnull().sum() * 100 / len(retail_df),2),"%" )

Summary..
Number of transactions:  13698
Number of products bought:  4088
Number of customers: 3529
Percentage of customers NA:  0.0 %


# RFM Analysis

## Recency

In [8]:
#last date available in our dataset
retail_uk['InvoiceDate'].max()

'9-30-2011    9:45'

In [9]:
now = dt.date(2011,12,10)
print(now)

2011-12-10


In [10]:
#create a new column called date which contains the date of invoice only
retail_uk['date'] = pd.DatetimeIndex(retail_uk['InvoiceDate']).date
retail_uk.head()

,sl_no,Invoice,StockCode,Description,Quantity,InvoiceDate,Date,Time,Price,Customer ID,Country,Total_price,gender,Age,date
91053,91053,501133,22382,LUNCH BAG SPACEBOY DESIGN,10,3-14-2010 10:10,14-03-2010,10:10,1.65,17920,United Kingdom,16.50,female,56,2010-03-14
91054,91054,501133,22385,JUMBO BAG SPACEBOY DESIGN,10,3-14-2010 10:10,14-03-2010,10:10,1.95,17920,United Kingdom,19.50,female,56,2010-03-14
91055,91055,501133,15044A,PINK PAPER PARASOL,2,3-14-2010 10:10,14-03-2010,10:10,2.95,17920,United Kingdom,5.90,female,56,2010-03-14
91056,91056,501133,15044C,PURPLE PAPER PARASOL,1,3-14-2010 10:10,14-03-2010,10:10,2.95,17920,United Kingdom,2.95,female,56,2010-03-14
91057,91057,501133,15044D,RED PAPER PARASOL,1,3-14-2010 10:10,14-03-2010,10:10,2.95,17920,United Kingdom,2.95,female,56,2010-03-14


In [11]:
#group by customers and check last date of purshace
recency_df = retail_uk.groupby(by='Customer ID', as_index=False)['date'].max()
recency_df.columns = ['Customer ID','LastPurshaceDate']
recency_df.head()

,Customer ID,LastPurshaceDate
0,12346,2010-06-30
1,12745,2010-05-14
2,12746,2010-06-30
3,12747,2011-08-22
4,12748,2011-09-30


In [12]:
#calculate recency
recency_df['Recency'] = recency_df['LastPurshaceDate'].apply(lambda x: (now - x).days)
recency_df.head()

,Customer ID,LastPurshaceDate,Recency
0,12346,2010-06-30,528
1,12745,2010-05-14,575
2,12746,2010-06-30,528
3,12747,2011-08-22,110
4,12748,2011-09-30,71


In [13]:
#drop LastPurchaseDate as we don't need it anymore
recency_df.drop('LastPurshaceDate',axis=1,inplace=True)

## Frequency

In [14]:
#drop duplicates
retail_uk_copy = retail_uk
retail_uk_copy.drop_duplicates(subset=['Invoice', 'Customer ID'], keep="first", inplace=True)
#calculate frequency of purchases
frequency_df = retail_uk_copy.groupby(by=['Customer ID'], as_index=False)['Invoice'].count()
frequency_df.columns = ['Customer ID','Frequency']
frequency_df.head()

,Customer ID,Frequency
0,12346,2
1,12745,1
2,12746,2
3,12747,11
4,12748,79


## Monetary

In [15]:
#create column total cost
retail_uk['TotalCost'] = retail_uk['Quantity'] * retail_uk['Price']

In [16]:
monetary_df = retail_uk.groupby(by='Customer ID',as_index=False).agg({'TotalCost': 'sum'})
monetary_df.columns = ['Customer ID','Monetary']
monetary_df.head()

,Customer ID,Monetary
0,12346,12.67
1,12745,76.32
2,12746,26.80
3,12747,580.78
4,12748,3378.72


## Create RFM Table

In [17]:
#merge recency dataframe with frequency dataframe
temp_df = recency_df.merge(frequency_df,on='Customer ID')
temp_df.head()

,Customer ID,Recency,Frequency
0,12346,528,2
1,12745,575,1
2,12746,528,2
3,12747,110,11
4,12748,71,79


In [18]:
#merge with monetary dataframe to get a table with the 3 columns
rfm_df = temp_df.merge(monetary_df,on='Customer ID')
#use CustomerID as index
rfm_df.set_index('Customer ID',inplace=True)
#check the head
rfm_df.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346,528,2,12.67
12745,575,1,76.32
12746,528,2,26.80
12747,110,11,580.78
12748,71,79,3378.72


## RFM Table Correctness verification

In [19]:
retail_uk[retail_uk['Customer ID']=='12820']

,sl_no,Invoice,StockCode,Description,Quantity,InvoiceDate,Date,Time,Price,Customer ID,Country,Total_price,gender,Age,date,TotalCost


In [20]:
(now - dt.date(2011,9,26)).days == 75

True

## Customer segments with RFM Model

### RFM Quartiles

In [21]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,Monetary
0.25,101.0,1.0,16.50
0.50,180.0,2.0,40.44
0.75,468.0,4.0,112.35


In [22]:
quantiles.to_dict()

{'Recency': {0.25: 101.0, 0.5: 180.0, 0.75: 468.0},
 'Frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 4.0},
 'Monetary': {0.25: 16.5, 0.5: 40.44, 0.75: 112.35}}

In [23]:
### Creation of RFM Segments

In [24]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [25]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

In [26]:
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile
Customer ID,,,,,,
12346,528,2,12.67,1,2,1
12745,575,1,76.32,1,1,3
12746,528,2,26.80,1,2,2
12747,110,11,580.78,3,4,4
12748,71,79,3378.72,4,4,4


In [27]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
Customer ID,,,,,,,
12346,528,2,12.67,1,2,1,121
12745,575,1,76.32,1,1,3,113
12746,528,2,26.80,1,2,2,122
12747,110,11,580.78,3,4,4,344
12748,71,79,3378.72,4,4,4,444


In [28]:
rfm_segmentation[rfm_segmentation['RFMScore']=='444'].sort_values('Monetary', ascending=False).head(10)

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
Customer ID,,,,,,,
18102,73,68,41437.99,4,4,4,444
17949,71,57,38850.10,4,4,4,444
17450,71,34,25696.32,4,4,4,444
16013,71,30,13215.34,4,4,4,444
15769,79,22,9603.39,4,4,4,444
13694,72,66,9328.01,4,4,4,444
16029,81,43,8010.32,4,4,4,444
12931,101,26,7575.77,4,4,4,444
15061,71,47,7436.11,4,4,4,444


### How many customers do we have in each segment?

In [29]:
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

Best Customers:  287
Loyal Customers:  827
Big Spenders:  882
Almost Lost:  63
Lost Customers:  19
Lost Cheap Customers:  302
